<a href="https://colab.research.google.com/github/AnnaFattakhova/poetry-llm-additional/blob/main/syllable_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

from datasets import load_dataset
import re
import pandas as pd
import unicodedata
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
import json
import numpy as np

In [ ]:
#1 — стримим датасет
# ссылка: https://huggingface.co/datasets/missvector/multi-wiki-grammar
def streaming():
    dataset = load_dataset("missvector/multi-wiki-grammar")
    df = dataset["train"].to_pandas()
     # Обрезаем до первых 1000 строк
    df = df.head(1000)
    return df

In [ ]:
# Загружаем для просмотра
df = streaming()
df.head()

In [ ]:
df.tail()

In [ ]:
# Здесь есть проблема: Й заменяется на И
# чистим, оставляем только слова на русском
# [a-яА-Я]
#def cleaning(text):
#    if pd.isna(text):
#        return text
    # Одним regex удаляем всё, кроме русских букв и пробелов + чистим пробелы
#    cleaned_text = re.sub(r'[^а-яА-ЯёЁ\s]+|\s+', ' ', str(text)).strip()
#    return cleaned_text if cleaned_text else pd.NA

In [ ]:
#2 — чистим датасет
# Пробуем нормализацию и проверку на тип данных, чтобы не терять Й
def cleaning(text):
    if pd.isna(text):
        return text
    if not isinstance(text, str):
        return text
    # Нормализуем в NFC (каноническая форма)
    text = unicodedata.normalize("NFC", text)
    cleaned_text = re.sub(r'[^а-яА-ЯёЁ\s]+|\s+', ' ', text).strip()
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text if cleaned_text else pd.NA

In [ ]:
df['Grammatical Sentences'].apply(lambda x: [cleaning(string) for string in x])

In [ ]:
df['Grammatical Sentences'].dropna().to_list

In [ ]:
# 3 — создаем список сникальных слов и дублируем его (основа для разделения на слоги)

# Функция для получения уникальных слов
def get_unique_words(sents):
    # Объединяем все строки столбца в один текст
    all_text = ' '.join()

    # Преобразуем текст в нижний регистр и токенизируем текст
    tokens = word_tokenize(all_text.lower())

    # Сохраняем уникальные слова
    unique_words = set(tokens)
    unique_words_array = np.array(list(unique_words))
    return unique_words_array

In [ ]:
# Получаем уникальные слова для всего столбца
unique_words = get_unique_words(df)

# Добавляем массив NumPy как новый столбец в DataFrame
# Поскольку список уникальных слов может быть длинным, подгоняем его под размер DataFrame
df_unique_words = pd.DataFrame(list(unique_words), columns=['Words'])

df_unique_words

In [ ]:
# Создаем второй столбец как копию первого
df_unique_words["Hyphenated_Words"] = df_unique_words["Words"]
# теперь ко второму столбцу надо применить токенизацию по слогам

In [ ]:
#3 — разделение на слоги
def syllables(text):
    vowel = 'аеёиоуыэюя'
    cons = 'бвгджзлмнрсхцчшщ'
    brief = 'кпстф'
    voiced = 'й'
    deaf = 'ьъ'
    other = 'бвгджзйклмнпрсстфхцчшщ'

    def _is_not_last_sep(txt):
        return any(ch in vowel for ch in txt)

    def _add_sep(force_sep):
        nonlocal current_syllable, syllables
        if force_sep:
            current_syllable += ' '
            return

        if not re.search(f'[{vowel}]', current_syllable):
            return

        syllables.append(current_syllable)
        current_syllable = ''

    syllables = []
    current_syllable = ''
    words = text.split()

    for word in words:
        if len(word) < 2:
            word = word.replace('-', '')

        for i, char in enumerate(word):
            current_syllable += char
            next_char = word[i+1] if i+1 < len(word) else ''

            if not next_char or next_char not in 'абвгдеёжзийклмнопрстуфхцчшщыьэюя':
                continue

            if (i != 0 and i != len(word)-1 and char in voiced and
                _is_not_last_sep(word[i+1:])):
                _add_sep(False)
                continue

            if (i < len(word)-1 and char in vowel and
                next_char in vowel):
                _add_sep(False)
                continue

            if (i < len(word)-2 and char in vowel and
                word[i+1] in other and word[i+2] in vowel):
                _add_sep(False)
                continue

            if (i < len(word)-2 and char in vowel and
                word[i+1] in brief and word[i+2] in other and
                _is_not_last_sep(word[i+1:])):
                _add_sep(False)
                continue

            if (i > 0 and i < len(word)-1 and char in cons and
                word[i-1] in vowel and next_char not in vowel and
                next_char not in deaf and
                _is_not_last_sep(word[i+1:])):
                _add_sep(False)
                continue

            if (i < len(word)-1 and char in deaf and
                (next_char not in vowel or _is_not_last_sep(word[:i]))):
                _add_sep(False)
                continue

        _add_sep(True)

    if current_syllable:
        syllables.append(current_syllable)

    return '-'.join(syllables)

In [ ]:
# Применяем функцию ко всему столбцу "Words" в df_unique_words
df_unique_words['Hyphenated_Words'] = df_unique_words['Words'].apply(syllables)

# Результат
df_unique_words['Hyphenated_Words']

In [ ]:
 #5 – создаем словарь,  где ключи - слова из первого столбца, а значения - из второго, и сохраняем в формате json
def save(df_unique_words):
    words_dict = dict(zip(df_unique_words["Hyphenated_Words"], df_unique_words["Words"]))

    with open("hyphenated_words_dict.json", "w", encoding="utf-8") as json_file:
        json.dump(words_dict, json_file, ensure_ascii=False, indent=4)

    return words_dict

In [ ]:
# Проверка результата
save(df_unique_words)

**Делим на слоги датасет из Викисловаря**

In [ ]:
!wget data.txt https://raw.githubusercontent.com/vifirsanova/stat-llm/refs/heads/main/morphs_output.txt

with open("morphs_output.txt", "r", encoding="utf-8") as morphs:
    words = morphs.read().strip().split('\n')

In [ ]:
words

In [ ]:
import pandas as pd
word_series = pd.Series(words, name = 'words')
word_series

In [ ]:
# Создаём DataFrame с двумя одинаковыми столбцами
df = pd.DataFrame({
    'words': word_series,
    'hyphenated_words': word_series  # дубликат
})
df

In [ ]:
# Разделение на слоги
import re
def syllables(text):
    vowel = 'аеёиоуыэюя'
    cons = 'бвгджзлмнрсхцчшщ'
    brief = 'кпстф'
    voiced = 'й'
    deaf = 'ьъ'
    other = 'бвгджзйклмнпрсстфхцчшщ'

    def _is_not_last_sep(txt):
        return any(ch in vowel for ch in txt)

    def _add_sep(force_sep):
        nonlocal current_syllable, syllables
        if force_sep:
            current_syllable += ' '
            return

        if not re.search(f'[{vowel}]', current_syllable):
            return

        syllables.append(current_syllable)
        current_syllable = ''

    syllables = []
    current_syllable = ''
    words = text.split()

    for word in words:
        if len(word) < 2:
            word = word.replace('-', '')

        for i, char in enumerate(word):
            current_syllable += char
            next_char = word[i+1] if i+1 < len(word) else ''

            if not next_char or next_char not in 'абвгдеёжзийклмнопрстуфхцчшщыьэюя':
                continue

            if (i != 0 and i != len(word)-1 and char in voiced and
                _is_not_last_sep(word[i+1:])):
                _add_sep(False)
                continue

            if (i < len(word)-1 and char in vowel and
                next_char in vowel):
                _add_sep(False)
                continue

            if (i < len(word)-2 and char in vowel and
                word[i+1] in other and word[i+2] in vowel):
                _add_sep(False)
                continue

            if (i < len(word)-2 and char in vowel and
                word[i+1] in brief and word[i+2] in other and
                _is_not_last_sep(word[i+1:])):
                _add_sep(False)
                continue

            if (i > 0 and i < len(word)-1 and char in cons and
                word[i-1] in vowel and next_char not in vowel and
                next_char not in deaf and
                _is_not_last_sep(word[i+1:])):
                _add_sep(False)
                continue

            if (i < len(word)-1 and char in deaf and
                (next_char not in vowel or _is_not_last_sep(word[:i]))):
                _add_sep(False)
                continue

        _add_sep(True)

    if current_syllable:
        syllables.append(current_syllable)

    return '-'.join(syllables)

In [ ]:
df['hyphenated_words'] = df['hyphenated_words'].apply(lambda x: syllables(x))
df

In [ ]:
df.to_csv('/content/syllables.csv', index=False, encoding='utf-8')